In [4]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter


In [3]:
# Load the data
file_path = Path('../Classification/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


In [5]:
# Create our features
X = df.drop(columns="homeowner")
X = pd.get_dummies(df.drop('loan_status', axis = 1))

# Create our target
y = df["loan_status"]
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
0,10700.0,7.672,52800,0.431818,5,1,22800,0,1,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0,1,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0,0,1
3,10700.0,7.664,52700,0.430740,5,1,22700,0,1,0
4,10800.0,7.698,53000,0.433962,5,1,23000,1,0,0


In [6]:
 X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.497472,0.398911,0.103616
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.499997,0.489678,0.304764
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000,0.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000,0.000000,0.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000,1.000000,0.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000,1.000000,1.000000


In [7]:
# Check the balance of our target values
y.value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [8]:
# Create X_train, X_test, y_train, y_test

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)


In [9]:
# Scale the training and testing data using the StandardScaler from sklearn. Remember that when scaling the data, you only scale the features data (X_train and X_testing).

# Create the StandardScaler instance

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset

X_scaler = scaler.fit(X_train)

# Scale the training and testing data

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [57]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_predict = model.predict(X_test)
bas = balanced_accuracy_score(y_test, y_predict)
bas

0.9543211898288821

In [12]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_predict)

cm_smote = confusion_matrix(y_test, y_predict)
cm_df = pd.DataFrame(cm_smote, index=["High Risk", "Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"])
cm_df

,Predicted High Risk,Predicted Low Risk
High Risk,571,54
Low Risk,93,18666


In [13]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_predict))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.91      1.00      0.89      0.95      0.90       625
   low_risk       1.00      1.00      0.91      1.00      0.95      0.92     18759

avg / total       0.99      0.99      0.92      0.99      0.95      0.92     19384



In [14]:
# In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

# View the count of the target classes using Counter from the collections library.
# Use the resampled data to train a logistic regression model.
# Calculate the balanced accuracy score from sklearn.metrics.
# Print the confusion matrix from sklearn.metrics.
# Generate a classication report using the imbalanced_classification_report from imbalanced-learn.
# Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [15]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)
X_resamp, y_resamp = ros.fit_resample(X_train_scaled, y_train)
Counter(y_resamp)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [16]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

linreg_ros = LogisticRegression(solver='lbfgs', random_state=1)
linreg_ros.fit(X_resamp, y_resamp)

LogisticRegression(random_state=1)

In [58]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

y_predict_ros = linreg_ros.predict(X_test_scaled)

bas_ros = balanced_accuracy_score(y_test, y_predict_ros)
print(bas_ros)

0.9946414201183431


In [59]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

pd.DataFrame(
    confusion_matrix(y_test, y_predict),
    index=["High Risk", "Low Risk"],
    columns=["Positive", "Negative"]
)

,Positive,Negative
High Risk,571,54
Low Risk,93,18666


In [60]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_predict))


                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.91      1.00      0.89      0.95      0.90       625
   low_risk       1.00      1.00      0.91      1.00      0.95      0.92     18759

avg / total       0.99      0.99      0.92      0.99      0.95      0.92     19384



In [61]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

X_resamp_smote, y_resamp_smote = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter

from collections import Counter
Counter(y_resamp_smote)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [21]:
# Train the Logistic Regression model using the resampled data

logreg_smote = LogisticRegression(solver='lbfgs', random_state=1)
logreg_smote.fit(X_resamp_smote, y_resamp_smote)

LogisticRegression(random_state=1)

In [62]:
# Calculated the balanced accuracy score

y_predict_smote = logreg_smote.predict(X_test_scaled)
bas_smote = balanced_accuracy_score(y_test, y_predict_smote)
print(bas_smote)

0.9946680739911509


In [63]:
# Display the confusion matrix

cm_smote = confusion_matrix(y_test, y_predict_smote)
cm_df_smote = pd.DataFrame(cm_smote, index=["High Risk", "Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"])
cm_df_smote

,Predicted High Risk,Predicted Low Risk
High Risk,622,3
Low Risk,110,18649


In [64]:
# Print the imbalanced classification report

print(classification_report_imbalanced(y_test, y_predict_smote))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



In [68]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=1)
X_resamp, y_resamp = cc.fit_resample(X_train, y_train)

from collections import Counter

Counter(y_resamp)

Counter({'high_risk': 1875, 'low_risk': 1875})

In [70]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
logreg_cc = LogisticRegression(solver='lbfgs', random_state=1)
logreg_cc.fit(X_resamp, y_resamp)

LogisticRegression(random_state=1)

In [71]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_predict = logreg.predict(X_test)
balanced_accuracy_score(y_test, y_predict)

0.9828813049736127

In [72]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_predict)
cm_df = pd.DataFrame(cm, index=["High Risk", "Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"])
cm_df

,Predicted High Risk,Predicted Low Risk
High Risk,607,18
Low Risk,102,18657


In [33]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_predict))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.97      0.99      0.91      0.98      0.96       625
   low_risk       1.00      0.99      0.97      1.00      0.98      0.97     18759

avg / total       0.99      0.99      0.97      0.99      0.98      0.97     19384



In [ ]:
# In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

# View the count of the target classes using Counter from the collections library.
# Use the resampled data to train a logistic regression model.
# Calculate the balanced accuracy score from sklearn.metrics.
# Print the confusion matrix from sklearn.metrics.
# Generate a classication report using the imbalanced_classification_report from imbalanced-learn.
# Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [76]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)
X_resamp_se, y_resamp_se = smote_enn.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resamp_se)

Counter({'high_risk': 55645, 'low_risk': 55932})

In [77]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

logreg_se = LogisticRegression(solver='lbfgs', random_state=1)
logreg_se.fit(X_resamp_se, y_resamp_se)

LogisticRegression(random_state=1)

In [78]:
# Calculated the balanced accuracy score
y_predict_se = logreg_se.predict(X_test_scaled)
bas_se = balanced_accuracy_score(y_test, y_predict_se)
bas_se

0.9946680739911509

In [79]:
# Display the confusion matrix
cm_combined = confusion_matrix(y_test, y_predict_se)
cm_df_combined = pd.DataFrame(cm_combined, index=["High Risk", "Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"])
cm_df_combined

,Predicted High Risk,Predicted Low Risk
High Risk,622,3
Low Risk,110,18649


In [80]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_predict))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.97      0.99      0.91      0.98      0.96       625
   low_risk       1.00      0.99      0.97      1.00      0.98      0.97     18759

avg / total       0.99      0.99      0.97      0.99      0.98      0.97     19384



In [ ]:
# Which model had the best balanced accuracy score?
# SMOTEEN since it was closest to 1.0

# Which model had the best recall score?
# Each model had an avg/total of .99

# Which model had the best geometric mean score? 
# SMOTE with .99